### Pattern arguments for Enumerable collection checks

Enumerable#any?, all?, none?, and one? accept a pattern argument. [Feature #11286](https://bugs.ruby-lang.org/issues/11286)

In [1]:
['Dane', 'Joe', 'Pramod'].any?(/an/)

true

In [2]:
['Dane', 'Joe', 'Pramod'].any?(/[aeiou]{2}/)

true

In [3]:
[1, 10, 100].one?(5..50)

true

In [4]:
{x: 1, y: 2}.values.any?(2..9)

true

### keyword_init for Struct

Struct.new can create classes that accept keyword arguments. [Feature #11925](https://bugs.ruby-lang.org/issues/11925)

In [5]:
Hashbrowns = Struct.new(:potatoes, :salt, :ketchup, keyword_init: true)
mine = Hashbrowns.new(salt: true, ketchup: 'of course', potatoes: 'russet')
mine

#<struct Hashbrowns potatoes="russet", salt=true, ketchup="of course">

In [6]:
mine.to_h

{:potatoes=>"russet", :salt=>true, :ketchup=>"of course"}

Struct can serve as a glorified Hash. The named arguments are a whitelist: they aren't required, but nothing else is allowed.

In [7]:
begin
  yours = Hashbrowns.new(potatoes: 'yukon gold', gravy: 'wtf')
rescue ArgumentError
  puts 'Only whitelisted arguments are allowed'
end

Only whitelisted arguments are allowed


Keyword initialization works well with JSON parsing.

In [8]:
input = <<~JSON
  {
    "ketchup": "gross",
    "salt": false,
    "potatoes": "red"
  }
JSON
hers = Hashbrowns.new(JSON.parse(input))

#<struct Hashbrowns potatoes="red", salt=false, ketchup="gross">

The default `to_json` for a Struct isn't great because it starts with the string representation.

In [9]:
hers.to_json

"\"#<struct Hashbrowns potatoes=\\\"red\\\", salt=false, ketchup=\\\"gross\\\">\""

You can override that with something that is at least useful:

In [10]:
class Hashbrowns
  def to_json
    require 'json'
    self.to_h.to_json
  end
end

:to_json

In [11]:
hers.to_json

"{\"potatoes\":\"red\",\"salt\":false,\"ketchup\":\"gross\"}"

In [12]:
his = Hashbrowns.new(JSON.parse(hers.to_json))
his == hers

true

In [13]:
his.to_json == hers.to_json

true

Reminder that you can modify Struct values much like you would a Hash:

In [14]:
his[:ketchup]

"gross"

In [15]:
his[:ketchup] = "gourmet"
his.to_h

{:potatoes=>"red", :salt=>false, :ketchup=>"gourmet"}